In [1]:
import numpy as np
import pandas as pd
import argparse
from sklearn.tree import DecisionTreeClassifier, plot_tree, _tree
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
import pydotplus
from matplotlib import pyplot as plt
from labels import mac_to_label
import json
import math

In [2]:
Set1 = pd.read_csv('data.csv').values.tolist()
X = [i[0:3] for i in Set1]
Y =[i[3] for i in Set1]

In [3]:
predict_Yt = []
index=0
with open('compressed_tree.json', 'r') as file:
    data = json.load(file)
    classes = data["classes"]
    for x in X:
        counter = 0
        class_set = []
        paths_set = []
        for feature in reversed(data['layers']): #Have to reverse this list due to structure of the data.csv file and how it aligns with the compressed tree layers
            for node in data['layers'][feature]:
                if node['min'] is None:
                    if x[counter] <= node['max']:
                        class_set.append(node['classes'])
                        paths_set.append(node["paths"])
                        break #is this an issue?
                    else:
                        continue
                elif node['max'] is None:
                    if node['min'] < x[counter]:
                        class_set.append(node['classes'])
                        paths_set.append(node["paths"])
                        break #is this an issue?
                    else:
                        continue
                elif node['min'] < x[counter] and x[counter] <= node['max']:
                    class_set.append(node['classes'])
                    paths_set.append(node["paths"])
                    break #is this an issue?

            counter += 1
        result = set(class_set[0])
        paths = set(paths_set[0])
        for s in class_set[1:]:
            result.intersection_update(s)
        for s in paths_set[1:]:
            paths.intersection_update(s)

        #predict_Yt.append(list(result))
        #print(result)
        if len(result) == 1:
            prediction = list(result)[0]
            pred_class = classes[prediction]
            predict_Yt.append(pred_class)
        elif len(paths) == 1:
            print("here1")
            print(pd.read_csv('data.csv'))
            print(result)
            print(paths)
            print(x)
            print(Y[index])
            break
            predict_Yt.append(None)
        else:
            print("here2")
            print(pd.read_csv('data.csv'))
            print(result)
            print(paths)
            print(x)
            print(Y[index])
            break
            predict_Yt.append(None)
        
        index += 1

here1
         protocl    src    dst    classfication
0              6  40234   5228            other
1              6  40234   5228            other
2              6    443  46330          Dropcam
3              6   3063    443            other
4              1      0      0   Netatmo Camera
...          ...    ...    ...              ...
2419339        6    443  47940          Dropcam
2419340        6  47940    443            other
2419341        6    443  47940          Dropcam
2419342        0      0      0  iHome PowerPlug
2419343        0      0      0            other

[2419344 rows x 4 columns]
{8, 20}
{13}
[6, 40234, 5228]
other


In [4]:
correct = 0
for i in range(len(Y)):
    prediction = predict_Yt[i]
    if prediction != None and Y[i] == prediction:
        correct += 1

print(correct / len(Y))

IndexError: list index out of range